Utilizando o Python para criar um script de carregamento para os clientes

In [2]:
import pandas as pd

Mostrando o conteudo do CSV (Apenas os dois primeiros)

In [2]:
df = pd.read_csv('dim_clientes.csv')
df.head(2)

,id_cliente,nome,segmento,id_regiao_sede
0,1,Montadora Alpha,OEM,NAM-US-MI
1,2,Distribuidora Beta,Revenda,SAM-BR-SP


Vamos primeiro entender os metadados do nosso arquivo, para checagem de tipo de dados etc

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id_cliente      8 non-null      int64 
 1   nome            8 non-null      object
 2   segmento        8 non-null      object
 3   id_regiao_sede  8 non-null      object
dtypes: int64(1), object(3)
memory usage: 388.0+ bytes


Agora vamos ao que interessa, a criação do script para inserção de dados no nosso SQL

Pensei de fazer de dois modos, um mais seguro e manual, e outro mais automatico

O primeiro modo eu um script que gera um arquivo SQL com os comandos de insert, e o segundo modo é um script que faz a inserção diretamente no banco de dados.

In [9]:
with open('inserts_clientes.sql', 'w', encoding='utf-8') as f:
    for _, row in df.iterrows():
        nome = row['nome'].replace("'", "''")
        segmento = row['segmento'].replace("'", "''")
        pais = row['id_regiao_sede'].replace("'", "''")
        
        insert_sql = (
            "INSERT INTO clientes (id_cliente, nome, segmento, pais) "
            "VALUES ({}, '{}', '{}', '{}');\n".format(row['id_cliente'], nome, segmento, pais)
        )
        f.write(insert_sql)

print("Arquivo 'inserts_clientes.sql' gerado com sucesso!")


Arquivo 'inserts_clientes.sql' gerado com sucesso!


In [4]:
# Primeiramente, vou utilizar uma biblioteca para conectar ao MySQL e inserir os dados do DataFrame.

import mysql.connector

# Função para inserir dados no MySQL, definindo o nome da tabela como 'clientes' e a conexão com o banco de dados.
def insert_data_to_mysql(df, table_name='clientes'):
    connection_params = {
        'host': 'localhost',
        'user': 'root',
        'password': 'pw4dummy',
        'database': 'prod_nsk'
    }
    data_to_insert = []  # declarar fora do try para retornar depois

    try:
        # Conectar ao banco de dados MySQL
        conn = mysql.connector.connect(**connection_params)
        # Criar um cursor para executar comandos SQL
        cursor = conn.cursor()

        # SQL para inserir dados na tabela
        # Usando f-string para facilitar a formatação
        sql = f"INSERT INTO {table_name} (id_cliente, nome, segmento, id_regiao_sede) VALUES (%s, %s, %s, %s)"

        for _, row in df.iterrows():
            values = (
                int(row['id_cliente']),
                row['nome'],
                row['segmento'],
                row['id_regiao_sede']
            )
            data_to_insert.append(values)

        cursor.executemany(sql, data_to_insert)
        conn.commit()

        #mensagem de sucesso
        print(f"{cursor.rowcount} registros inseridos com sucesso na tabela {table_name}.")
    
    except mysql.connector.Error as err:
        # mensagem de erro
        print(f"Erro MySQL: {err}")
    
    # fechar o cursor e a conexão
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()
    
    return data_to_insert  # retornando aqui

# Executar e capturar os dados
data_to_insert = insert_data_to_mysql(df)


8 registros inseridos com sucesso na tabela clientes.
